Dyson Sphere Program received an interesting update called the [Icarus
Evolution][icarus-notes]. One of the more intriguing parts of the update is the
completely new system of "sprayers" and "proliferators," whereby benefits to
production can be achieved if all inputs into a manufacturing process are
"sprayed," sort of like some sort of lubricant. It is somewhat reminiscent of
Factorio's [productivity][fact-prod] or [speed][fact-speed] modules. That is,
for production, you can produce extra outputs from a factory (either 12.5%, 20%,
or 25% extra), given that all the inputs for that production are "sprayed." (At
least provisionally people are calling the proliferator spray "paint," and
referring to the act of passing through the sprayer "painting.")

[icarus-notes]: https://dsp-wiki.com/Patch_Notes/0.9.24.11182
[fact-prod]: https://wiki.factorio.com/Productivity_module_3
[fact-speed]: https://wiki.factorio.com/Speed_module_3

To give an example of how this can work, the lowest tier paint (yellow paint)
provides the 12.5% production bonus, and takes one coal to make. (This marks the
first recipe that takes coal other than energized graphite, as point of
interest.) So with the simplest production, 1 coal provides 1 yellow paint.

Except, of course, you can loop some of the output of this process back to paint
the inputs. So if you take some output and boost its own production, it provides
12.5% bonus, so for every 8 production cycles, it actually produces 9 units of
paint. Or, equivalently, and as I think more usefully for the purpose of
analysis, for every 9 units produced, it only consumed 8 units.

Simultaneously, the yellow paint can be applied to 12 items, before a new unit
of paint begins to be consumed -- or, again perhaps more usefully, each unit
being painted consumes 1/12th of the yellow paint. So you are not quite getting
a 12.5% bonus straight up on input, but rather that 12.5% minus the loss you get
for applying the paint. What does that come out to?

In [1]:
# One output unit of yellow paint requires 1/1.125 input coal.
coal = 1/1.125
# However, we also consumed some yellow paint, which is `coal/12` units, so the
# effective output is really 1 minus that.
effective_yellow = 1 - coal/12
# The rate of actual bonus is then actually the ratio between produced divided
# by input coal.
bonus = effective_yellow/coal - 1

print(f"Effective bonus production: {bonus:.4%}")
print(f"Coal consumed per output: {coal/effective_yellow}")

Effective bonus production: 4.1667%
Coal consumed per output: 0.96


In understanding how to minimize coal consumption, it's obvious even now that
things are definitely more complicated than the simple ratios we enjoyed in the
game until this update. Indeed, until I ran the simple analysis above, it was
very unclear what the actual costs of production were for producing the paints,
even in this simple case of yellow paint only.

But, what would happen if we were to use green paint instead? Without any
production enhancement at all, that costs four total input coal, instead of the
one for yellow, so it seems like it might not be worth it, at least for this
specific production chain. But, the situation is not quite so simple as that,
either, because with the application of green paint on the input stages.

We've mentioned diamond, so let's have an aside about kimberlite. In a star
system with lots of coal and some kimberlite and the spiniform crystals, making
the green paint from kimberlite makes sense. If we view kimberlite as basically
trash, then you might as well just use it where readily available to get the
benefits of green paint for "free." Nonetheless it is worthwhile to consider
what to do once local reserves of kimberlite are exhausted before the coal is
exhausted.

Our goal is resource efficiency, in this case coal. That said, some of the
recipes don't need to just take coal, or coal derived products. So, let's be
explicit about our assumptions:

1. Kimberlite is not readily accessible and plentiful, since if it were, then
   you may as well use it to create green paint and use it.
2. We are not using energetic graphite from X-ray cracking. (Ever since oil well
   exhaustion was introduced in KSP, I have never found occassion to use
   cracking.) Any graphite would come from coal.
3. We will not use blue paint on the yellow/green paint production chain, since
   spending carbon nanotubes to stretch out reserves of coal or kimberlite is
   silly, considering the relatively prevalence of coal/kimberlite vs. spiniform
   stalagmite.
4. As a corollary, if we discuss blue paint, we assume we want blue paint
   applied to its production steps, since that results in a net savings of the
   nanotubes.

Let's now think about using green paint at all stages:

In [2]:
# First set the consumption rates, e.g., coal per yellow, coal per graphite, etc.,
# assuming green paint.
gp = 1.2
coal_yellow, coal_graph, graph_diamond, yellow_green, diamond_green = 1/gp, 2/gp, 1/gp, 2/gp, 1/gp
# For one unit of green paint, we are consuming `yellow_green` amount of yellow paint,
# and `diamond_green` amount of diamonds, but these products have inputs, which in turn
# might have their own inputs, which also need to be sprayed.
coal_y_green = coal_yellow * yellow_green
graph_d_green = graph_diamond * diamond_green
coal_gd_green = coal_graph * graph_d_green
# All of the `_green` chain consumptions account for that many "sprays" of green paint itself.
total_sprays = yellow_green + diamond_green + coal_y_green + graph_d_green + coal_gd_green
print(f"Production of one green requires {total_sprays:.4f} sprays.")
# Calculate the effective green by discounting how many sprays.
effective_green = 1 - total_sprays / 24
# The total amount of actual coal consumed per effective output will give us a
# feeling for the efficiency.
coal_green = coal_y_green + coal_gd_green
print(f"Coal consumed per green: {coal_green/effective_green:.4f}")

Production of one green requires 5.7407 sprays.
Coal consumed per green: 3.3469


This becomes a bit more interesting. The production of one green paint (which
itself consists of 24 sprays) will consume almost a fourth of those sprays, and
the total amount of coal consumed to produce a net output of a single green is
considerably better than if we used no sprays at all.

One thing possibly interesting to consider is if we don't spray one of the
inputs, that is, if it goes in "raw," say, coal to graphite.

In [3]:
# Let's run that again, but let's assume that the energized graphite is produced "raw"
# from coal.
raw_coal_graph = 2.0
raw_coal_gd_green = raw_coal_graph * graph_d_green
raw_total_sprays = total_sprays - coal_gd_green
raw_effective_green = 1 - raw_total_sprays / 24
raw_coal_green = coal_y_green + raw_coal_gd_green
print(f"Coal consumed per green if graphite is raw: {raw_coal_green/raw_effective_green:.4f}")


Coal consumed per green if graphite is raw: 3.4335


We see that not applying the green spray to the coal resulted in an inefficiency. However, is any tradeoff possible? There are a few steps in this pipeline, and while we've evaluated what happens if we apply green paint to all of them (a cost of ~3.3 coal), and we know what happens if we apply paint to none of them (a cost of exactly 4 coal), I do not yet have confidence that *no* mixture strategy of yellow or none on some stages is not appropriate.

In [4]:
# Here we compare the effective "bonus" per coal given the above. This is a
# fairly problematic comparison for several reasons, but if the yellow had been
# lower than the green then I would have suspected it was never beneficial to
# apply yellow paint, ever, if one could have green.
print(f"Effective yellow bonus per coal: {effective_yellow/coal * 12 * 0.125:.4f}")
print(f"Effective green bonus per coal: {effective_green/coal_green * 24 * 0.2:.4f}")

Effective yellow bonus per coal: 1.5625
Effective green bonus per coal: 1.4342


In [5]:
# Paint bonus productivity improvement factors (with index 0 meaning no paint
# applied at all.)
pp = [1.0, 1.15, 1.20, 1.25]
# Paint item count per tier.
pc = [1, 12, 24, 60]
# Energy consumption factor.
pe = [1.0, 1.3, 1.7, 2.5]

While these one-off analyses are interesting in their own right, something that
examines a mixed approach of using perhaps green here, yellow there, or even no
paint in some circumstances is fairly more complex.

Let's assign the symbols $c$, $g$, $d$ to coal, energized graphite, and diamond
respectively, and $x$, $y$, $z$ to no paint, yellow paint, and green paint. (As
stated above, we are not considering blue paint for these production tabs.)

One thing that helps us a little bit is that each intermediate product is used
as an ingredient in only one downstream recipe, so we can use something like
$p_g$ to mean the productivity improvement of whatever paint is applied to (in
this case) the ingredient(s) of graphite (one of $\{1.0, 1.15, 1.20\}$), whereas
$s_g$ might mean the total number of sprays of whatever paint is applied to the
ingredient(s) of graphite.

We can also say something like $g_d$ is the graphite going into the diamond
production, $c_g$ is the coal going into graphite. So, for example, $c_g = 2
\cdot g_d / p_g$, which means, that coal going into graphite, is twice the
number of graphite going into the diamond (which happens to be all graphite),
then divided by whatever productivity bonus we have.



In [6]:
import sympy

# We have:
# c_g : Coal going into graphite.
# c_y : Coal into yellow paint.
# g_d : Graphite into diamond.
# y_z : Yellow paint into green.
# d_z : Diamond into green.

# y_s : Yellow paint sprays. This equals the sum of all inputs being accelerated using yellow paint.
# z_s : Green paint sprays. This equals the sum of all inputs being accelerated using green paint.

# p_g, p_d, p_y, p_z is the productivity of each of the items (must be one of 1.0, 1.125, 1.2).
# s_g, s_d, s_y, x_z is the corresponding spray values for the paint being used (respective to above, 1, 12, 24).

# The goal is to find the configuration of what is sprayed on what so as to minimize c_g + c_y, 

cg, cy, gd, yz, dz, ys, zs = sympy.symbols('cg cy gd yz dz ys zs')

# The productivity/spray things set per item.
pg, pd, py, pz = sympy.symbols('pg pd py pz')
sg, sd, sy, sz = sympy.symbols('sg sd sy sz')

# c : Total coal, just c_g + c_y.
# y : Total yellow paint. y = y_z + y_s / 12.
# z : Total green paint, with the invariant held that we're targetting a net output of 1.
#     That is: z = 1 + z_s / 24.
c, y, z = sympy.symbols('c y z')

# Come up with the base equations, in the form of an expression that should evluate to zero.

exprs = []

# Total coal, yellow, and green.
exprs.append(c - cg - cy)
exprs.append(y - yz - ys/12)
exprs.append(z - 1 - zs/24)
# The graphite for the diamond production, depends on 2 coals to graphite.
exprs.append(cg - 2*gd/pg)
# The diamond for green production, depends on the graphite.
exprs.append(gd - dz/pd)
# The yellow production all up depends upon coal for yellow.
exprs.append(cy - y/py)
# Green production depends upon 2 yellows for green, and also the diamonds.
exprs.append(yz - 2*z/pz)
exprs.append(dz - z/pz)

all_vars = [c, cg, cy, y, ys, yz, z, zs, gd, dz]

def spray_regime(gi, di, yi, zi):
    'Returns a (modified) version of the global expressions.'
    # These temporary variables hold the amount of sprays necessary.
    temp = [ys, zs]
    replacements = []

    def extensions(i, p, s, *t):
        if i==0: # None on this stage.
            replacements.extend( [(p, 1), (s, 1)] )
        elif i==1: # Yellow on this stage.
            replacements.extend( [ (p, 1.125), (s, 12)] )
            for tt in t:
                temp[0] = temp[0] - tt
        elif i==2: # Green on this stage.
            replacements.extend( [(p, 1.2), (s, 24)] )
            for tt in t:
                temp[1] = temp[1] - tt

    extensions(gi, pg, sg, cg) # Graphite depends on coal for graphite.
    extensions(di, pd, sd, gd) # Diamond depends on graphite for diamond.
    extensions(yi, py, sy, cy) # Yellow depends on coal for yellow.
    extensions(zi, pz, sz, yz, dz) # Green depends on yellow and diamond for green.

    #print(f"FOO {replacements}")

    new_exprs = [e.subs(replacements) for e in exprs] + temp
    return new_exprs

def coal_for_regime(gi, di, yi, zi):
    e = spray_regime(gi, di, yi, zi)
    # Solve the system of linear equations. There *must* be a better way to solve a system of linear equations, return the result for one variable of interest,
    # but unfortunately I can't quite find it.
    return float(list(sympy.solvers.linsolve(e, all_vars))[0][0])

In [7]:
# Let's first look that the non-accelerated vanilla pipeline equations look correct.
for e in spray_regime(0, 0, 0, 0): print(e)

c - cg - cy
y - ys/12 - yz
z - zs/24 - 1
cg - 2*gd
-dz + gd
cy - y
yz - 2*z
dz - z
ys
zs


In [8]:
# This should be 4.
coal_for_regime(0,0,0,0)

4.0

In [9]:
# From our previous work, using greens everywhere should be about 3.3469.
coal_for_regime(2,2,2,2)

3.3468559837728193

In [10]:
# While using greens for everything, except the graphite, should be about 3.4335.
coal_for_regime(0,2,2,2)

3.433476394849788

In [18]:
# Substituting the spray on the graphite production and taking it off diamond
# should look a bit worse than even that, since we are now 
coal_for_regime(2,0,2,2)

3.560830860534125

In [11]:
ii = range(3)
# Great! Now let's just manually find each minimum regime.
min((coal_for_regime(gi,di,yi,zi),gi,di,yi,zi) for gi in ii for di in ii for yi in ii for zi in ii)

(3.3468559837728193, 2, 2, 2, 2)

Well, that was a bit anticlimatic: when doing a coal-only building of greens,
it's best to just pipe the green paint onto all the segments leading up to the
production of the green paint, and the ~3.3469 coal per green is in fact the
best we can do (without kimberlite and blue painting the inputs, of course). As
discussed above I don't consider using blue paint, since in my experience coal
is far, far more plentiful throughout the star cluster as compared to spiniform
stalagmite crystals.